In [18]:
def read_file():
    file = open('data1/short_1_dummy.fasta', 'r')
    reads = []
    lines = file.readlines()
    for line in lines:
        if not line.startswith(">"):
            reads.append(line.strip())
    file.close()
    return reads


In [19]:
def read_to_kmers(reads, k):
    kmers_list = {}
    for read in reads:
        for i in range(len(read)-k+1):
            kmer = read[i:i+k]
            if kmer not in kmers_list.keys():
                kmers_list[kmer] = 1
            else:
                kmers_list[kmer] += 1
    return kmers_list


In [20]:
def get_debruijn_edges_from_kmers(kmers):
    # store edges as tuples in a set
    edges = set()

    # compare each (k-1)mer
    for k1 in kmers:
        for k2 in kmers:
            if k1 != k2:
                # if they overlap then add to edges
                if k1[1:] == k2[:-1]:
                    edges.add((k1[:-1], k2[:-1]))
                if k1[:-1] == k2[1:]:
                    edges.add((k2[:-1], k1[:-1]))

    return edges


In [21]:
import toyplot


def plot_debruijn_graph(edges, width=700, height=700):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">",
        vsize=25,
        vstyle={"stroke": "white", "stroke-width": 8, "fill": "white"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "white", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph


In [22]:
k = int(input("Enter the value of k : "))
reads = read_file()
kmers = read_to_kmers(reads, k)
edges = get_debruijn_edges_from_kmers(kmers)
plot_debruijn_graph(edges)


(<toyplot.canvas.Canvas at 0x2c76b959d30>,
 <toyplot.mark.Graph at 0x2c76b96efa0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="700.0px" viewBox="0 0 700.0 700.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tab8d58bd00cf43c681b468be6829ba4b"> AAA AAC AAG AAT ACA ACC ACG ACT AGA AGC AGG AGT ATA ATC ATG ATT CAA CAC CAG CAT CCA CCC CCG CCT CGA CGC CGG CGT CTA CTC CTG CTT GAA GAC GAG GAT GCA GCC GCG GCT GGA GGC GGG GGT GTA GTC GTG GTT TAA TAC TAG TAT TCA TCC TCG TCT TGA TGC TGG TGT TTA TTC TTG TTT

In [23]:
def get_dbg_from_edges(edges):
    graph = {}
    for edge in edges:
        if edge[0] not in graph:
            graph[edge[0]] = [edge[1]]
        else:
            graph[edge[0]].append(edge[1])

    return (graph)


graph = get_dbg_from_edges(edges)


In [26]:
def bfs(graph, node, k):
    visited = []  # List for visited nodes.
    queue = []  # Initialize a queue

    BFS_traversal = []

    visited.append(node)
    queue.append(node)
    contig = ""

    while queue:          # Creating loop to visit each node
        m = queue.pop(0)
        BFS_traversal.append(m)

        for neighbour in graph[m]:

            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return " -> ".join(BFS_traversal)


In [29]:
print("BFS traversal of the de Bruijn Graph is: ")
print(bfs(graph, list(graph.keys())[0], k))


BFS traversal of the de Bruijn Graph is: 
CGT -> GTC -> GTA -> GTT -> GTG -> TCC -> TCT -> TCG -> TCA -> TAT -> TAA -> TAG -> TAC -> TTG -> TTA -> TTT -> TTC -> TGA -> TGC -> TGG -> TGT -> CCG -> CCC -> CCA -> CCT -> CTG -> CTC -> CTA -> CTT -> CGC -> CGG -> CGA -> CAC -> CAG -> CAT -> CAA -> ATG -> ATA -> ATT -> ATC -> AAA -> AAT -> AAG -> AAC -> AGT -> AGC -> AGA -> AGG -> ACA -> ACT -> ACC -> ACG -> GAT -> GAC -> GAA -> GAG -> GCA -> GCT -> GCC -> GCG -> GGG -> GGT -> GGC -> GGA
